---
title: "IFQ720 Worksheet 08 - Diagnostics"
output:
  html_document
---

In [ ]:
import pandas
import plotnine
import numpy
import scipy
import math
from plotnine import *


# Diagnostics

All models, including those discussed so far, ANOVA, linear regression, and logisitic regression, rely on some assumptions.  Model Diagnositics are a set of tools to checks the underlying model assumptions, including if there are any other exogenous effects not accounted for in the model. 

## Diagnostic Tools for Testing the Regression Results

Let's consider the model from Module 6 where we fit a simple linear regression model for the city and highway mileages from the `epa_data` dataset. 

Recall that the relationship between highway fuel economy ($x$) and
city fuel economy ($y$) is
$$
y_i = \beta_0 + \beta_1x_i.
$$

The underlying assumptions about the distribution of the residuals and
hence the sampling distributions of the parameter estimates have
important implications for both inference and, as we will see, using the
regression model for prediction. Luckily there are several useful tools
that rely on using the residuals
$$
e_i =y_i-\hat{\beta}_0-\hat{\beta}_1
$$ 
available for exploring and testing the validity of these assumptions.

###  Normality  {#normality .unnumbered}

The first assumption to check is that the residuals from the model are
from a Gaussian distribution. We can do this using a normal probability
plot. A normal probability plot is a plot of the residuals against their
expected value if they had come from a Gaussian distribution. If the
residuals are from a Gaussian distribution, they should fall more
or less along a straight, diagonal line. We plot this is by first
sorting the residuals in ascending order, then standardising them
$$e_i^* = \frac{e_i}{\sqrt{s^2}}$$ and plotting them against their
corresponding quantiles from a standard Gaussian distribution.

The use of this plot is not a rigorous test of normality versus
non-normality, but instead can indicate potential deviance from the
Gaussian assumption, particularly in the behaviour of the tails of the
distribution. The results are not conclusive but should inform
judgement when interpreting the model results.

### Heteroskedasticity {#heteroskedasticity .unnumbered}

The second residual plot we can use is to plot the fitted values of the
model 
$$
\hat{y}_i = \hat{\beta}_0+\hat{\beta}_1x_i
$$ 
against the residuals. The residuals should fall evenly along either side of a
horizontal line centred at $0$, with no evidence of trend or increase in
magnitude as $\hat{y}$ increases. We use this plot to check the assumption that
the variance of the residuals is equal for all observations, and
specifically that the variance is not a function of the expected
value of the observation.

The plots for the residuals versus fitted values and the $q-q$ plots show that (despite the small sample size) the residuals
appear to adhere to the model's assumptions. The variance (i.e. the
magnitude of the residuals) doesn't appear to be a function of the
fitted value, indicating that the variance of the residuals appears
constant. The $q-q$ plot looks approximately Gaussian (supported by the
histogram as well). We can disregard the index plot for now, but it also
supports the notion of homoskedasticity in the residuals.


### Other concerns {#other-concerns .unnumbered}

The two techniques of using normal probability plots and plots of the
residuals versus the fitted values address two of the three assumptions
about the residuals: the unbiasedness of the regression model and
the constant variance of the residuals. Tests of independence are
technically possible and, while beyond the scope of this material, should
be used in cases where the independent variables are temporally or
spatially referenced. The specific tools for modelling time-series or
spatial data exist, and we should use them where appropriate.

###   Example
Analyse the results for the model
$$
city = \beta_0+\beta_1hwy
$$
Do the assumptions for linear regression appear to be met?

####    Solution

The methods we want to check are graphical, meaning that they rely on our interpretation of visual plots.  Hence, they are qualitative and not statistically rigorous, and somewhat subjective.  Still, they are useful and relatively straightforward to understand.  If needed, there are more rigorous methods for formally testing the regression assumptions. 
\
\
In general, we rely on computational tools for creating the images needed.

####    Code

In [ ]:

import statsmodels

df = pandas.read_csv("../DATA/epa_data.csv")

df = df[df["year"]==1999]

model = statsmodels.formula.api.ols('city~hwy',df)

result = model.fit()

residuals = result.resid

fitted_values = result.fittedvalues

df_fit = pandas.DataFrame({'residuals':residuals,'fitted_values':fitted_values})

plot1 = (ggplot(df,aes(x = 'hwy', y = 'city'))+
  geom_point()+
    geom_smooth(method = "lm")).draw();

plot2 = (ggplot(df_fit,aes(x = 'residuals'))+
  geom_histogram(bins = 21)).draw();

plot3 = (ggplot(df_fit,aes(x = 'fitted_values',y = 'residuals'))+
  geom_point()).draw();

plot4 = (ggplot(df_fit,aes(sample='residuals'))+
  geom_qq()+
    geom_qq_line()).draw();


####    Plot

In [ ]:
plot1

In [ ]:
plot2

In [ ]:
plot3

In [ ]:
plot4

The first plot shows that the linear relationship between city and highway full economy is reasonable.  The histogram of the residuals and the $q$-$q$ plot show that the residuals are approximately Gaussian.  The plot of the residuals versus the fitted values shows some evidence of heteroskedasticity, which doesn't look extreme but is likely as the full economy measures are based on counts (i.e. Poisson-like), indicating that the variance is proportional to the expected value.

####    Improvement

Note that there are some areas of concern, and in particular the residuals versus fitted plot shows some clear outliers. One way we can address this is by looking at adding additional variable to the model.

In practice you might try different variable and decide on which ones to include in your model by trial and error (though the study of systematic and rigourous ways to do this are quite extensive).  But in our case we are going to select the number of cylinders in the engine (`cyl`) as our additional variable just to see what if that can reduce the outliers and "clean up" the results

In [ ]:

result_2 = statsmodels.formula.api.ols('city~hwy+cyl',df).fit()

df_fit = pandas.DataFrame({'residuals_2':result_2.resid,'fitted_values_2':result_2.fittedvalues})

plot1 = (ggplot(df,aes(x = 'hwy', y = 'city'))+
  geom_point()+
    geom_smooth(method = "lm")).draw();

plot2 = (ggplot(df_fit,aes(x = 'residuals_2'))+
  geom_histogram(bins = 21)).draw();

plot3 = (ggplot(df_fit,aes(x = 'fitted_values_2',y = 'residuals_2'))+
  geom_point()).draw();

plot4 = (ggplot(df_fit,aes(sample='residuals_2'))+
  geom_qq()+
    geom_qq_line()).draw();

In [ ]:
plot1

In [ ]:
plot2

In [ ]:
plot3

In [ ]:
plot4

We can note that the second model residuals appear to be much better behaved.  While there are still some outliers, they aren't as extreme as in the first model and the residual versus fitted values plot looks more like what we would like to see.  

Note that finally we can compare the $R^2_{adj}$ values to compare model and pick the "better" one. We note that the second model (including `cyl`) is the preferred model under this criteria. 

##    Diagnostic Tools for ANOVA Models

ANOVA and ANCOVA models rely on a similar set of assumptions as linear regerssion models, but they also have some additional areas of concern that need to be addressed. We will look at some data for the salaries for IT professionals to understand how to diagnose ANOVA models and address some issues that might arise. 

Our data is an example Salary Table for IT professionals with varying educational backgrounds (Bachelors, Masters, and PhD), experience (Years), and level (Management or Lower).  First let's explore the data, noting that Experience in years is a continuous variable which we are going to set aside for now. 

In [ ]:
#S = Salary 
#X = Experience
#E = Education
#P = Managment or Lower Level

df = pandas.read_csv("../DATA/salary.csv")

df = df.rename(columns = {'S': 'Salary', 'X': 'Experience','E': 'Education','P': 'Level'})

ed = (
  ggplot(df)+
  geom_boxplot(aes(y = 'Salary', x = 'Education'))
).draw();


level = (
  ggplot(df)+
  geom_boxplot(aes(y = 'Salary', x = 'Level'))
).draw();

In [ ]:
ed

In [ ]:
level

Note that we are interested in the relationship between Salary and Education, 

In [ ]:

import statsmodels

model = statsmodels.formula.api.ols('Salary ~ C(Education)+C(Level)',df).fit()

aov_model = statsmodels.stats.anova.anova_lm(model)

print(aov_model)





It appears that both education and your level are statistically significant. Look at the summary of the OLS model and see that 

In [ ]:

model.summary()


This looks a little odd, there is not a statistically significant difference between Bachelors and PhD, this seems counter intuitive. 

Let's check the residuals

In [ ]:

df["res"] = model.resid
df['ind'] = df.index

(ggplot(df)+
geom_point(aes(y = 'res', x = 'ind'))

)

res = (ggplot(df)+geom_qq(aes(sample = 'res'))+
geom_qq_line(aes(sample = 'res'))
).draw();

res


Uh-Oh this looks very odd, the residuals aren't centered at 0 and instead are linearly related to the order of observations?  Seems like our model might be missing something, perhaps we ought to look at experience. This is an example of an ANCOVA model and we need to check to see if there is an interaction between the other variables.

In [ ]:
ex_ed = (
  ggplot(df)+
  geom_point(aes(y = 'Salary', x = 'Experience', color = 'Education'))
).draw();

ex_lev = (
  ggplot(df)+
  geom_point(aes(y = 'Salary', x = 'Experience', color = 'Level'))
).draw();

In [ ]:
ex_ed

In [ ]:
ex_lev

The data for experience versus salary appear to be parallel for Education and Level, so it appears that the assumption for ANCOVA is valid. 

In [ ]:

model_2 = statsmodels.formula.api.ols('Salary ~ C(Education)+C(Level)+Experience',df).fit()

aov_model_2 = statsmodels.stats.anova.anova_lm(model_2)

print(aov_model_2)

model_2.summary()



In [ ]:
df["resid_2"] = model_2.resid

df['ind'] = df.index

res2_1 = (ggplot(df)+
geom_point(aes(y = 'resid_2', x = 'ind'))+ylim(-9000,9000)
).draw();

res2_2 = (ggplot(df)+geom_qq(aes(sample = 'resid_2'))+
geom_qq_line(aes(sample = 'resid_2'))+ylim(-9000,9000)
).draw();

In [ ]:
res2_1

In [ ]:
res2_2

This looks a better.  The residuals now appear unbiased  and the range of variables is much smaller.  The `q-q` plot looks a bit better too.  As for linear models we can alos compare the $R^2_{adj}$ values as well to make some judgement about model performance and preference. 

##    For Further Thought

In this module we have barely scratched the surface of tools and techniques for evaluating models.  There are numerous tools including information criteria for model selection, stepwise procedures for variable selections, and cross validation for testing the predictive capabilities of a model.   

We should also note that for the logistic regression model the diagnostics become much more complicated and their derivation and explanation are beyond the scoe of this course.  It is our advice that you should consult other resources or seek out expert advice if you have any other questions or concerns when fitting a logistic regression model.  

#   Further Examples

## Additional Questions for Practice

1.    The concentration of asbestos fibres is important to environmental health. There are two methods for measuring the concentration of small asbestos fibres: scanning electron microscope (SEM) and phase-contrast microscope (PCM).  Are the two methods equivalent?  Assume that the SEM results are a "gold standard".  What do your results tell you about how the PCM compares to SEM?  What aspects of the model should you look at to determine this? The data are in the dataset `asbestos`  

In [ ]:
df = pandas.read_csv("../DATA/asbestos.csv")



In [ ]:

df = pandas.read_csv("../DATA/asbestos.csv")

model = statsmodels.formula.api.ols("PCM~SEM",df)

results = model.fit()

results.summary()


Now, perform diagnostics on the model residuals and comment on the results.

**Solution:**

In [ ]:
df = pandas.read_csv("../DATA/asbestos.csv")

model = statsmodels.formula.api.ols("PCM~SEM",df)

results = model.fit()

results.summary()

df["fittedvalues"] = results.fittedvalues
df["residuals"] = results.resid

plot1 = (ggplot(df,aes(x = 'SEM', y = 'PCM'))+
  geom_point()+
    geom_smooth(method = "lm")).draw();

plot2 = (ggplot(df,aes(x = 'residuals'))+
  geom_histogram(bins = 21)).draw();

plot3 = (ggplot(df,aes(x = 'fittedvalues',y = 'residuals'))+
  geom_point()).draw();

plot4 = (ggplot(df,aes(sample='residuals'))+
  geom_qq()+
    geom_qq_line()).draw();
    

In [ ]:
plot1

In [ ]:
plot2

In [ ]:
plot3

In [ ]:
plot4

1.    The scatter plot looks odd with the "bulge" in the middle.
2.    Histogram of residuals doesn't look symmetric (i.e. not Gaussian)
3.    Residuals vs Fitted Values looks off again "bulge" in middle. 
4.    Q-Q Plot looks off too, errors are probably not Gaussian, appear herteroskedastic.  Need to consider more looking into things.

2.    A proving ring is a device for measuring load based on deflection.  The data set `provingring` results include the deflection for several know loads and three separate repetitions (runs).  Fit a linear model relating the deflection to the load.  Is there any evidence of hysteresis?  Examine the model results and the residuals, is there any way to improve the results, e.g.\ a way to more accurately predict the deflection from the load?

In [ ]:
df = pandas.read_csv("../DATA/provingring.csv")



In [ ]:
df = pandas.read_csv("../DATA/provingring.csv")
  
model = statsmodels.formula.api.ols("Deflection~Load",df)

results = model.fit()

results.summary()


Now, perform diagnostics on the model residuals and comment on the results.

**Solution:**

In [ ]:
df = pandas.read_csv("../DATA/provingring.csv")

model = statsmodels.formula.api.ols("Deflection~Load",df)

results = model.fit()

results.summary()

df["fittedvalues"] = results.fittedvalues
df["residuals"] = results.resid

plot1 = (ggplot(df,aes(x = 'Load', y = 'Deflection'))+
  geom_point()+
    geom_smooth(method = "lm")).draw();

plot2 = (ggplot(df,aes(x = 'residuals'))+
  geom_histogram(bins = 21)).draw();

plot3 = (ggplot(df,aes(x = 'fittedvalues',y = 'residuals'))+
  geom_point()).draw();

plot4 = (ggplot(df,aes(sample='residuals'))+
  geom_qq()+
    geom_qq_line()).draw();
    

In [ ]:
plot1

In [ ]:
plot2

In [ ]:
plot3

In [ ]:
plot4

1.    Scatterplot looks good, a little "too" good.
2.    Historgram does not look Gaussian
3.    Residuals versus Fitted Values has a parabolic shape, we should add a quadratic term.
4.    Q-Q Plot looks a bit off, there appears to be censoring in the measurements, definitely not Gaussian.  

In [ ]:
df = pandas.read_csv("../DATA/provingring.csv")
df["Load2"] = df.Load**2

model = statsmodels.formula.api.ols("Deflection~Load+Load2",df)

results = model.fit()

results.summary()

df["fittedvalues"] = results.fittedvalues
df["residuals"] = results.resid

plot1 = (ggplot(df,aes(x = 'Load', y = 'Deflection'))+
  geom_point()+
    geom_smooth(method = "lm")).draw();

plot2 = (ggplot(df,aes(x = 'residuals'))+
  geom_histogram(bins = 21)).draw();

plot3 = (ggplot(df,aes(x = 'fittedvalues',y = 'residuals'))+
  geom_point()).draw();

plot4 = (ggplot(df,aes(sample='residuals'))+
  geom_qq()+
    geom_qq_line()).draw();
    

In [ ]:
plot1

In [ ]:
plot2

In [ ]:
plot3

In [ ]:
plot4

1.    Histogram now appears bi-modal
2.    Resdiuals versus Fitted Values looks better, but still "bulgy"
3.    Q-Q Plot looks better but still looks like censored measurements.  